# Image_Recognition_with_RNNs

In this tutorial, we are going to learn how to use RNNs to predict numbers through images.

Source:
- hands on machine learning

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


It is important to list all the `attibutes` and the `methods` in a unknown `dataset object`. So we use the `dir()` method to get access to the dataset object. we fould that there is a `train` attribute in `mnist class`. Under the `train` attribute, there are `images`, `labels`, `num_examples` attributes.

In [2]:
print(dir(mnist))
print(dir(mnist.train))
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
print(mnist.train.num_examples)

print(mnist.test.labels)

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_fields', '_make', '_replace', '_source', 'count', 'index', 'test', 'train', 'validation']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_epochs_completed', '_images', '_index_in_epoch', '_labels', '_num_examples', 'epochs_completed', 'images', 'label

We have 55000 images and each image is composed of `28x28 pixels`. What we are going to do is use each `row` in the image as a sequence, and the will be `28 time steps`. In other words, each time step, there are `28 inputs`. Each sequence will be fed in a cell with `150 neurons`. The only output will be at the `final cell`, where the 150 neurons* 150 neurons array would be fed in a softmax that preoduces `10 outputs` predicting the image from 0 to 9.

Note that in this case, we are only going to feed the last output (one could also say it is the last state because in vanilla RNN, the last state  equals the output), which is the second term in the (outputs, states) tuple.

#### What is a softmax function?
The softmax function squeeses the input arguments into a valus between 0 and 1.
$$ \sigma(x_j) = \frac{e^{x_j}}{\Sigma_i e^{x_i}} $$

#### What is Cross entropy loss?
$$ H(p,q) = -\Sigma_x p(x)logq(x) $$
where p is the observed value and q is the estimated value.

- Why use `tf.nn.sparse_softmax_cross_entropy_with_logits` to calculate the cross entropy?
We are working on a multiclass calssification problem where the catogirical data is 0-9. If we use `softmax_cross_entropy_with_logits`, we have to transform the catogorical data to one hot vector. However, with `tf.nn.sparse_softmax_cross_entropy_with_logits`, it take cares fo the one-hot encoding precedure.

Source:
1. https://stackoverflow.com/questions/37312421/whats-the-difference-between-sparse-softmax-cross-entropy-with-logits-and-softm
2. https://stackoverflow.com/questions/43394152/tensorflow-what-exact-formula-is-applied-in-tf-nn-sparse-softmax-cross-entropy

#### What is `tf.nn.in_top_k(predictions, targets, k)`?

Let the index of the 0th to kth largest value in the prediction be a set M, then if targets is in the set M, then it is true.

Source:
- https://www.cnblogs.com/logo-88/p/9099383.html

In [3]:
import tensorflow as tf
tf.reset_default_graph() 
A = tf.Variable([[0.8, 0.4, 0.5, 0.6],[0.1, 0.9, 0.2, 0.4],[0.1, 0.9, 0.4, 0.2]])
B = tf.Variable([1, 1, 1])
result = tf.nn.in_top_k(A, B, 2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(A))
    print(sess.run(B))
    print(sess.run(result))

import tensorflow as tf
tf.reset_default_graph() 
A = tf.Variable([[0.8, 0.4, 0.5, 0.6],[0.1, 0.9, 0.2, 0.4],[0.1, 0.9, 0.4, 0.2]])
B = tf.Variable([1, 1, 2])
result = tf.nn.in_top_k(A, B, 2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(A))
    print(sess.run(B))
    print(sess.run(result))

[[0.8 0.4 0.5 0.6]
 [0.1 0.9 0.2 0.4]
 [0.1 0.9 0.4 0.2]]
[1 1 1]
[False  True  True]
[[0.8 0.4 0.5 0.6]
 [0.1 0.9 0.2 0.4]
 [0.1 0.9 0.4 0.2]]
[1 1 2]
[False  True  True]


`correct = tf.nn.in_top_k(logits, y, 1)` means that is the largest value in logits is equal to y, then it is true.
Why?
Because if logit looks like this,
0.1 0.2 0.0 0.8 0.4 0.6 0.0 0.1 0.1 0.3
the index of the largest value in logits is 3, it also means that the predictor thinks that the image is 3
Now, if the real value y is 3, then `correct = tf.nn.in_top_k(logits, 3, 1)` returns True.
if the real value y is 5, then `correct = tf.nn.in_top_k(logits, 5, 1)` returns False.

In [4]:
import tensorflow as tf
tf.reset_default_graph() 
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
learning_rate = 0.001
n_epochs = 10
batch_size = 150


X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
#states is the final output
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#             print(states.eval(feed_dict={X: X_batch, y: y_batch}).shape)
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
0 Train accuracy: 0.96 Test accuracy: 0.9312
1 Train accuracy: 0.96 Test accuracy: 0.9578
2 Train accuracy: 0.97333336 Test accuracy: 0.9614
3 Train accuracy: 0.9866667 Test accuracy: 0.9621
4 Train accuracy: 0.96 Test accuracy: 0.9583
5 Train accuracy: 0.96666664 Test accuracy: 0.9539
6 Train accuracy: 0.97333336 Test accuracy: 0.972
7 Train accuracy: 0.98 Test accuracy: 0.9657
8 Train accuracy: 0.9533333 Test accuracy: 0.9635
9 Train accuracy: 0.9866667 Test accuracy: 0.9691
